In [47]:
import re
from collections import defaultdict
from scipy.stats import norm
import itertools

import pyagrum as gum
import pyagrum.lib.notebook as gnb

In [48]:
# Create CN
bn=gum.fastBN("A[2]->B[2]<-C[2]->D[2]->E[2]")
bn_min=gum.BayesNet(bn)
bn_max=gum.BayesNet(bn)
for n in bn.nodes():
  x=0.4*min(bn.cpt(n).min(),1-bn.cpt(n).max())
  bn_min.cpt(n).translate(-x)
  bn_max.cpt(n).translate(x)

cn=gum.CredalNet(bn_min,bn_max)
cn.intervalToCredal()

In [49]:
import io
from contextlib import redirect_stdout

# Creo un buffer in memoria
buffer = io.StringIO()

# Uso il contesto per reindirizzare stdout al buffer
with redirect_stdout(buffer):
    print(cn)

# Recupero il contenuto del buffer
output = buffer.getvalue()

credal_text = output

In [50]:
def parse_credal_net(cn_str: str):
    """
    Parsa l'output testuale di una credal net da PyAgrum.
    Restituisce un dizionario del tipo:
        {
            'A': {'<>': [ [p1], [p2], ... ]},
            'B': {'<A:0|C:0>': [...], ...},
            ...
        }
    """
    credal_dict = defaultdict(lambda: defaultdict(list))
    current_var = None

    lines = cn_str.strip().split('\n')

    for line in lines:
        line = line.strip()

        # Identificazione della variabile
        var_match = re.match(r'^([A-Za-z0-9_]+):Range\(\[.*\]\)', line)
        if var_match:
            current_var = var_match.group(1)
            continue

        if current_var is None or not line:
            continue

        # Identificazione di una CPT con intestazione <condizioni>
        cpt_match = re.match(r'^<([^>]*)>\s*:\s*(.*)', line)
        if cpt_match:
            condition = f"<{cpt_match.group(1).strip()}>"
            raw_cpt = cpt_match.group(2)

            # Estrarre tutte le liste interne: [[x,x,x], [x,x,x], ...]
            vectors = re.findall(r'\[\s*([^\[\]]+?)\s*\]', raw_cpt)
            for vec in vectors:
                prob_list = [float(x.strip()) for x in vec.split(',')]
                credal_dict[current_var][condition].append(prob_list)

    return credal_dict


In [51]:
# # Debug
# parsed = parse_credal_net(credal_text)
# for var in parsed:
#     print(f"--- {var} ---")
#     for cond in parsed[var]:
#         print(f"{cond} : {parsed[var][cond]}")


**Notice**: up to now, this code only works with binary variables

In [52]:
def generate_all_bayesian_nets(credal_dict):
    # 1. Domini
    var_domains = {var: len(next(iter(cpts.values()))[0]) for var, cpts in credal_dict.items()}

    # 2. Ricaviamo struttura (genitori per ogni variabile)
    parent_dict = {var: set() for var in var_domains}
    for var, conds in credal_dict.items():
        for cond in conds:
            if cond == "<>":
                continue
            matches = re.findall(r'([A-Za-z0-9_]+):([0-9]+)', cond)
            for parent, _ in matches:
                parent_dict[var].add(parent)

    # 3. Costruiamo una rete base vuota
    template_bn = gum.BayesNet()
    var_ids = {}
    for var, domain_size in var_domains.items():
        var_ids[var] = template_bn.add(gum.LabelizedVariable(var, var, domain_size))
    for var, parents in parent_dict.items():
        for p in parents:
            template_bn.addArc(var_ids[p], var_ids[var])

    # 4. Lista di CPT (var, cond, vettori)
    slots = []
    for var in credal_dict:
        for cond, vectors in credal_dict[var].items():
            slots.append((var, cond, vectors))

    # 5. Tutte le combinazioni possibili
    all_combinations = list(itertools.product(*[vecs for _, _, vecs in slots]))

    # Debug: Mostra il numero di combinazioni e la prima combinazione
    print(f"Numero di combinazioni totali: {len(all_combinations)}")
    # print(f"Prima combinazione (cpt): {all_combinations[0]}")

    nets = []
    for combo in all_combinations:
        # 6. Creiamo una nuova rete per ogni combinazione
        bn = gum.BayesNet()  # Rete nuova
        new_var_ids = {}

        # Ricreiamo la struttura della rete (archi e variabili)
        for var, domain_size in var_domains.items():
            new_var_ids[var] = bn.add(gum.LabelizedVariable(var, var, domain_size))
        for var, parents in parent_dict.items():
            for p in parents:
                bn.addArc(new_var_ids[p], new_var_ids[var])

        # 7. Assegnamo le CPT alle variabili nella nuova rete
        for slot_idx, (var, cond, _) in enumerate(slots):
            vec = combo[slot_idx]  # Prendiamo il vettore associato alla combinazione corrente

            # Debug: Mostra cosa stiamo assegnando
            # print(f"Assegnando CPT a {var} con condizione {cond}: {vec}")

            t = bn.cpt(var)

            if cond == "<>":
                # Nessun genitore
                for i, p in enumerate(vec):
                    t[i] = p
            else:
                # Es: <A:1|C:0> → A=1, C=0
                cond_vals = dict(re.findall(r'([A-Za-z0-9_]+):([0-9]+)', cond))
                cond_vals = {k: int(v) for k, v in cond_vals.items()}

                parent_order = [bn.variable(p).name() for p in bn.parents(bn.idFromName(var))]
                indices = [cond_vals.get(p, 0) for p in parent_order]

                # Ogni combinazione di indici dei genitori per la variabile figlia
                # Aggiungiamo l'indice della variabile figlia (per i valori della variabile)
                for i, p in enumerate(vec):
                    # Creiamo l'indice completo che tiene conto di tutte le dimensioni
                    try:
                        t[tuple(indices + [i])] = p
                    except:
                        # Aggiungi un debug informativo se si verifica un errore di indicizzazione
                        print(f"Errore di indicizzazione per {var} con condizione {cond} e indici {indices + [i]}")
                        raise

        nets.append(bn)

    # Debug: Mostra il numero finale di reti generate
    print(f"Numero di reti generate: {len(nets)}")
    return nets


In [53]:
# Passaggio 1: parsing
parsed = parse_credal_net(credal_text)

# Passaggio 2: generazione delle reti bayesiane
bayesian_nets = generate_all_bayesian_nets(parsed)


Numero di combinazioni totali: 1024
Numero di reti generate: 1024


In [54]:
def are_all_bn_different(bn_list):
    """
    Controlla se tutte le reti bayesiane nella lista sono diverse
    (basandosi sulla serializzazione delle loro CPT).
    """
    def serialize_bn(bn):
        cpt_data = []
        for var in bn.names():
            cpt = bn.cpt(var)
            flat = [f"{v:.8f}" for v in cpt.toarray().flatten()]
            cpt_data.append(f"{var}:" + ",".join(flat))
        return "|".join(cpt_data)

    signatures = set()
    for bn in bn_list:
        sig = serialize_bn(bn)
        signatures.add(sig)

    return len(signatures) == len(bn_list)


In [55]:
if are_all_bn_different(bayesian_nets):
    print("✅ Tutte le reti sono diverse tra loro.")
else:
    raise Exception("⚠️ Alcune reti sono identiche.")


✅ Tutte le reti sono diverse tra loro.


In [56]:
def filter_bns(bn_list):
    """
    Ritorna una lista con solo le reti bayesiane uniche, rimuovendo i duplicati.
    """
    def serialize_bn(bn):
        cpt_data = []
        for var in bn.names():
            cpt = bn.cpt(var)
            flat = [f"{v:.8f}" for v in cpt.toarray().flatten()]
            cpt_data.append(f"{var}:" + ",".join(flat))
        return "|".join(cpt_data)

    unique_sigs = set()
    unique_bns = []

    for bn in bn_list:
        sig = serialize_bn(bn)
        if sig not in unique_sigs:
            unique_sigs.add(sig)
            unique_bns.append(bn)

    return unique_bns

In [57]:
# # Debug
# unique_nets = filter_bns(bayesian_nets)
# print(f"Reti dopo filtro: {len(unique_nets)}")


In [58]:
def generate_all_bayesian_nets_old(credal_dict):
    # 1. Domini
    var_domains = {var: len(next(iter(cpts.values()))[0]) for var, cpts in credal_dict.items()}

    # 2. Ricaviamo struttura (genitori per ogni variabile)
    parent_dict = {var: set() for var in var_domains}
    for var, conds in credal_dict.items():
        for cond in conds:
            if cond == "<>":
                continue
            matches = re.findall(r'([A-Za-z0-9_]+):([0-9]+)', cond)
            for parent, _ in matches:
                parent_dict[var].add(parent)

    # 3. Costruiamo una rete base vuota
    template_bn = gum.BayesNet()
    var_ids = {}
    for var, domain_size in var_domains.items():
        var_ids[var] = template_bn.add(gum.LabelizedVariable(var, var, domain_size))
    for var, parents in parent_dict.items():
        for p in parents:
            template_bn.addArc(var_ids[p], var_ids[var])

    # 4. Lista di CPT (var, cond, vettori)
    slots = []
    for var in credal_dict:
        for cond, vectors in credal_dict[var].items():
            slots.append((var, cond, vectors))

    # 5. Tutte le combinazioni possibili
    all_combinations = list(itertools.product(*[vecs for _, _, vecs in slots]))

    # Debug: Mostra il numero di combinazioni e la prima combinazione
    print(f"Numero di combinazioni totali: {len(all_combinations)}")  # Debug
    print(f"Prima combinazione (cpt): {all_combinations[0]}")  # Debug

    nets = []
    for combo in all_combinations:
        # 6. Creiamo una nuova rete per ogni combinazione
        bn = gum.BayesNet()  # Rete nuova
        new_var_ids = {}

        # Ricreiamo la struttura della rete (archi e variabili)
        for var, domain_size in var_domains.items():
            new_var_ids[var] = bn.add(gum.LabelizedVariable(var, var, domain_size))
        for var, parents in parent_dict.items():
            for p in parents:
                bn.addArc(new_var_ids[p], new_var_ids[var])

        # 7. Assegnamo le CPT alle variabili nella nuova rete
        for slot_idx, (var, cond, _) in enumerate(slots):
            vec = combo[slot_idx]  # Prendiamo il vettore associato alla combinazione corrente

            # Debug: Mostra cosa stiamo assegnando
            print(f"Assegnando CPT a {var} con condizione {cond}: {vec}")

            t = bn.cpt(var)

            if cond == "<>":
                # Nessun genitore
                for i, p in enumerate(vec):
                    t[i] = p
            else:
                # Es: <A:1|C:0> → A=1, C=0
                cond_vals = dict(re.findall(r'([A-Za-z0-9_]+):([0-9]+)', cond))
                cond_vals = {k: int(v) for k, v in cond_vals.items()}

                parent_order = [bn.variable(p).name() for p in bn.parents(bn.idFromName(var))]
                # parent_order = [t.variable(i).name() for i in range(t.nbrDim() - 1)]
                indices = [cond_vals.get(p, 0) for p in parent_order]

                for i, p in enumerate(vec):
                    try:
                        t[tuple(indices + [i])] = p
                    except:
                        raise ValueError(f"Index error for var {var} with indices {indices + [i]} in CPT")

        nets.append(bn)

    # Debug: Mostra il numero finale di reti generate
    print(f"Numero di reti generate: {len(nets)}")  # Debug
    return nets
